logistic回归是一种广义线性回归（generalized linear model），与多重线性回归分析有很多相同之处。它们的模型形式基本上相同，都具有 wx + b，其中w和b是待求参数，其区别在于他们的因变量不同，多重线性回归直接将wx+b作为因变量，即y =wx+b,而logistic回归则通过函数L将wx+b对应一个隐状态p，p =L(wx+b),然后根据p 与1-p的大小决定因变量的值。如果L是logistic函数，就是logistic回归，如果L是多项式函数就是多项式回归。

In [15]:
import torch
import torch.nn as nn
import numpy as np
torch.__version__

'1.1.0'

In [16]:
data=np.loadtxt("german.data-numeric")

In [17]:
n,l=data.shape
for j in range(l-1):
    meanVal=np.mean(data[:,j])
    stdVal=np.std(data[:,j])
    data[:,j]=(data[:,j]-meanVal)/stdVal

In [18]:
np.random.shuffle(data)

In [19]:
train_data=data[:900,:l-1]
train_lab=data[:900,l-1]-1
test_data=data[900:,:l-1]
test_lab=data[900:,l-1]-1

In [20]:
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc=nn.Linear(24,2) # 由于24个维度已经固定了，所以这里写24
    def forward(self,x):
        out=self.fc(x)
        out=torch.sigmoid(out)
        return out

In [21]:
def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

In [22]:

net=LR() 
criterion=nn.CrossEntropyLoss() # 使用CrossEntropyLoss损失
optm=torch.optim.Adam(net.parameters()) # Adam优化
epochs=1000 # 训练1000次

In [23]:

for i in range(epochs):
    # 指定模型为训练模式，计算梯度
    net.train()
    # 输入值都需要转化成torch的Tensor
    x=torch.from_numpy(train_data).float()
    y=torch.from_numpy(train_lab).long()
    y_hat=net(x)
    loss=criterion(y_hat,y) # 计算损失
    optm.zero_grad() # 前一步的损失清零
    loss.backward() # 反向传播
    optm.step() # 优化
    if (i+1)%100 ==0 : # 这里我们每100次输出相关的信息
        # 指定模型为计算模式
        net.eval()
        test_in=torch.from_numpy(test_data).float()
        test_l=torch.from_numpy(test_lab).long()
        test_out=net(test_in)
        # 使用我们的测试函数计算准确率
        accu=test(test_out,test_l)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.2f}".format(i+1,loss.item(),accu))

Epoch:100,Loss:0.6437,Accuracy：0.70
Epoch:200,Loss:0.6154,Accuracy：0.72
Epoch:300,Loss:0.5971,Accuracy：0.75
Epoch:400,Loss:0.5846,Accuracy：0.77
Epoch:500,Loss:0.5756,Accuracy：0.79
Epoch:600,Loss:0.5687,Accuracy：0.79
Epoch:700,Loss:0.5632,Accuracy：0.79
Epoch:800,Loss:0.5586,Accuracy：0.79
Epoch:900,Loss:0.5547,Accuracy：0.80
Epoch:1000,Loss:0.5514,Accuracy：0.80
